In [15]:
import pandas as pd

In [16]:


prices = pd.read_csv("PCOFFOTMUSDM.csv")

prices.rename(columns={"PCOFFOTMUSDM": "coffee_price"}, inplace=True)
prices["date"] = pd.to_datetime(prices["observation_date"])
prices = prices[["date", "coffee_price"]]

print(prices.head())


        date  coffee_price
0 1990-01-01     75.829987
1 1990-02-01     84.010010
2 1990-03-01     93.959991
3 1990-04-01     93.730011
4 1990-05-01     92.019989


In [17]:


faostat = pd.read_csv(r"C:\Users\hp\OneDrive\Belgeler\DSA210_Project\data\Production_Crops_Livestock_E_Africa.csv")
faostat.head()


,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,...,Y2020N,Y2021,Y2021F,Y2021N,Y2022,Y2022F,Y2022N,Y2023,Y2023F,Y2023N
0,4,'012,Algeria,221,'01371,"Almonds, in shell",5312,Area harvested,ha,13300.0,...,NaN,32492.0,A,NaN,38495.00,I,NaN,38544.00,I,NaN
1,4,'012,Algeria,221,'01371,"Almonds, in shell",5412,Yield,kg/ha,451.1,...,NaN,1706.5,A,NaN,1720.50,E,NaN,1806.70,E,NaN
2,4,'012,Algeria,221,'01371,"Almonds, in shell",5510,Production,t,6000.0,...,NaN,55448.0,A,NaN,66232.31,I,NaN,69637.53,I,NaN
3,4,'012,Algeria,515,'01341,Apples,5312,Area harvested,ha,3400.0,...,NaN,31171.0,A,NaN,32636.00,I,NaN,32149.00,I,NaN
4,4,'012,Algeria,515,'01341,Apples,5412,Yield,kg/ha,4529.4,...,NaN,16756.3,A,NaN,16834.00,E,NaN,17912.90,E,NaN


In [ ]:
import pandas as pd
import re


faostat = pd.read_csv(r"C:\Users\hp\OneDrive\Belgeler\DSA210_Project\data\Production_Crops_Livestock_E_Africa_NOFLAG.csv")


coffee = faostat[faostat["Item"] == "Coffee, green"]

coffee_prod = coffee[coffee["Element"] == "Production"]


year_cols = [col for col in coffee_prod.columns if re.fullmatch(r"Y\d{4}", col)]


coffee_long = coffee_prod.melt(
    id_vars=["Area", "Unit"],
    value_vars=year_cols,
    var_name="year_col",
    value_name="production_tons"
)

coffee_long["Year"] = coffee_long["year_col"].str[1:].astype(int)


coffee_long = coffee_long[["Area", "Year", "production_tons"]]
coffee_long.rename(columns={"Area": "country"}, inplace=True)

coffee_unique = coffee_long.groupby(["country", "Year"], as_index=False).mean()


coffee_unique["date"] = pd.to_datetime(coffee_unique["Year"], format="%Y")


coffee_monthly = (
    coffee_unique.set_index("date")
    .groupby("country")["production_tons"]
    .resample("M")
    .ffill()
    .reset_index()
)

coffee_monthly.head()


C:\Users\hp\AppData\Local\Temp\ipykernel_28080\2476952038.py:41: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


,country,date,production_tons
0,Angola,1961-01-31,168600.0
1,Angola,1961-02-28,168600.0
2,Angola,1961-03-31,168600.0
3,Angola,1961-04-30,168600.0
4,Angola,1961-05-31,168600.0
